In [3]:
import pandas as pd
import numpy as np
import torch.nn as nn

from numpy import vstack

from sklearn.preprocessing import  MinMaxScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor

from torch.optim import SGD
from torch.nn import BCELoss

from opacus import PrivacyEngine
from sklearn.metrics import accuracy_score

MAX_GRAD_NORM = 1.2
EPSILON = 50.0
DELTA = 1e-6
EPOCHS = 10

LR = 1e-3
BATCH_SIZE = 512


def accuracy(preds, labels):

    return (preds == labels).mean()


# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self):
        # Load dataset
        data = pd.read_csv('../scripts/lightpath_data.csv')
        data = data.loc[:, ~data.columns.isin(['sample','conn_id','src_id','dst_id'])]


        target = pd.read_csv('../scripts/lightpath_target.csv')
        target = target['class']
        data['target'] = target
        data = data.sample(100000)
        target = data['target']
        data = data.loc[:, data.columns!='target']
        target = np.array(target).reshape(len(target),1)
        data = data.values

        scaler = MinMaxScaler()
        scaler = scaler.fit(data)
        data = scaler.transform(data)

        self.X = data
        self.y = target
        self.X = self.X.astype('float32')
        self.y = self.y.astype('float32')

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])
# prepare the dataset
def prepare_data():
    # load the dataset
    dataset = CSVDataset()
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=BATCH_SIZE, shuffle=False)
    test_dl = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)
    return train_dl, test_dl

train_dl, test_dl = prepare_data()


# model definition
class neural_network(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(32, 64)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(64, 256)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(256, 1)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x

model = neural_network()
# ----------------------------------------------------
criterion = BCELoss()
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

# enter PrivacyEngine
privacy_engine = PrivacyEngine()
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dl,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)
# ----------------------------------------------------
print(f"Using sigma={optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")
# train the model
def train_model(model, train_loader, optimizer, epoch):
    model.train()
    criterion = nn.BCELoss()

    losses = []
    top1_acc = []

    for i, (inputs, target) in enumerate(train_loader):
        optimizer.zero_grad()

        # compute output
        output = model(inputs)
        loss = criterion(output, target)

        preds = np.argmax(output.detach().cpu().numpy(), axis=1)
        labels = target.detach().cpu().numpy()

        # measure accuracy and record loss
        acc = accuracy(preds, labels)

        losses.append(loss.item())
        top1_acc.append(acc)

        loss.backward()
        optimizer.step()
        epsilon = privacy_engine.get_epsilon(DELTA)
        if (i+1) % 200 == 0:
            # epsilon = privacy_engine.get_epsilon(DELTA)
            print(
                f"\tTrain Epoch: {epoch} \t"
                f"Loss: {np.mean(losses):.6f} "
                f"Acc@1: {np.mean(top1_acc) * 100:.6f} "
                f"(ε = {epsilon:.2f}, δ = {DELTA})"
            )
    return round(epsilon,2), round(np.mean(losses),2), round(acc,2)


from tqdm.notebook import tqdm

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    epsilon, loss, acc = train_model(model, train_loader, optimizer, epoch + 1)
    print(epoch , epsilon, loss, acc)

# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        yhat = yhat.round()
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    prec = precision_score(actuals, predictions)
    recc = recall_score(actuals, predictions)
    f1 = f1_score(actuals, predictions)

    return acc, prec, recc, f1

acc = evaluate_model(test_dl, model)


# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

c:\users\sander\desktop\env_pyth\lib\site-packages\opacus\privacy_engine.py:131: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "
c:\users\sander\desktop\env_pyth\lib\site-packages\opacus\accountants\analysis\rdp.py:333: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  f"Optimal order is the {extreme} alpha. Please consider expanding the range of alphas to get a tighter privacy bound."


Using sigma=0.3520774841308594 and C=1.2


Epoch:   0%|          | 0/10 [00:00<?, ?epoch/s]

c:\users\sander\desktop\env_pyth\lib\site-packages\torch\nn\modules\module.py:1117: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


0 23.13 0.62 0.25
1 28.1 0.61 0.22
2 31.52 0.61 0.25
3 34.94 0.62 0.24
4 37.86 0.62 0.26
5 40.29 0.63 0.25
6 42.72 0.63 0.24
7 45.14 0.63 0.25
8 47.57 0.63 0.24
9 50.0 0.64 0.27


In [4]:

acc, prec, recc, f1 = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)
print('Recall: %.3f' % prec)
print('Precision: %.3f' % recc)
print('Precision: %.3f' % f1)

c:\users\sander\desktop\env_pyth\lib\site-packages\torch\nn\modules\module.py:1117: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Accuracy: 0.758
Recall: 0.758
Precision: 1.000
Precision: 0.862
